In [10]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.monster.com/jobs/search/?q=Software-Developer&where=Australia'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find(id='ResultsContainer')
print(results.prettify())

<div class="mux-custom-scroll" data-extend="left" data-mux="customScroll" data-target="html" id="ResultsContainer">
 <div class="scrollable" id="ResultsScrollable">
  <script type="application/ld+json">
   {"@context":"https://schema.org","@type":"ItemList","mainEntityOfPage":{
            "@type":"CollectionPage","@id":"https://www.monster.com/jobs/search/?q=Software-Developer&amp;where=Australia"
            }
            ,"itemListElement":[

                 {"@type":"ListItem","position":1,"url":"https://job-openings.monster.com/sql-bi-ssrs-ssis-developer-for-blackboard-nyc-new-york-wa-us-lancesoft-inc/56d17f16-f07d-4271-abda-b80155837c80"}
                    ,
                 {"@type":"ListItem","position":2,"url":""}
                    ,
                 {"@type":"ListItem","position":3,"url":"https://job-openings.monster.com/python-developer-woodlands-wa-us-lancesoft-inc/4755ec59-d0db-4ce9-8385-b4df7c1e9f7c"}
                    ,
                 {"@type":"ListItem","positi

In [12]:
job_elems = results.find_all('section', class_='card-content')
for job_elem in job_elems:
    print(job_elem, end='\n'*2)


<section class="card-content" data-jobid="56d17f16-f07d-4271-abda-b80155837c80" onclick="MKImpressionTrackingMouseDownHijack(this, event)">
<div class="flex-row">
<div class="mux-company-logo thumbnail is-loaded">
<img alt="LanceSoft Inc" src="https://media.newjobs.com/clu/xlan/xlancesinx/branding/164991/LanceSoft-Inc-logo.png"/>
</div>
<div class="summary">
<header class="card-header">
<h2 class="title"><a data-bypass="true" data-m_impr_a_placement_id="JSR2CW" data-m_impr_j_cid="10" data-m_impr_j_coc="" data-m_impr_j_jawsid="359506666" data-m_impr_j_jobid="0" data-m_impr_j_jpm="2" data-m_impr_j_jpt="3" data-m_impr_j_lat="40.75" data-m_impr_j_lid="550" data-m_impr_j_long="-73.9967" data-m_impr_j_occid="11709" data-m_impr_j_p="1" data-m_impr_j_postingid="56d17f16-f07d-4271-abda-b80155837c80" data-m_impr_j_pvc="4496dab8-a60c-4f02-a2d1-6213320e7213" data-m_impr_s_t="t" data-m_impr_uuid="032e5eaf-79f5-4df5-981f-fa3d57aec50c" href="https://job-openings.monster.com/sql-bi-ssrs-ssis-developer

In [43]:
import requests
from bs4 import BeautifulSoup

URL = 'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports_us'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
results = str(soup.table.find_all('a')[-4]['href'])
print(type(results))

<class 'str'>


In [51]:

github = 'https://github.com'
URL = github + results

page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
raw = str(soup.find(id='raw-url')['href'])
print(raw)

/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-21-2020.csv


In [54]:
URL = github + raw
print(URL)
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
soup

https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports_us/04-21-2020.csv


Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
Alabama,US,2020-04-21 23:40:34,32.3182,-86.9023,5317,183,,5134.0,1,113.39553843899749,48526,699,3.4417904833552755,84000001,USA,1034.9129016909521,13.14651119052097
Alaska,US,2020-04-21 23:40:34,61.3707,-152.4044,329,9,168,320.0,2,55.03982440790365,11119,42,2.735562310030395,84000002,USA,1860.1453118282088,12.76595744680851
American Samoa,US,2020-04-21 23:40:34,-14.270999999999999,-170.132,0,0,,,60,0.0,3,,,16,ASM,5.3917075537822825,
Arizona,US,2020-04-21 23:40:34,33.7298,-111.4312,5256,208,1155,5048.0,4,72.21052831151424,55152,637,3.95738203957382,84000004,USA,757.7159546112316,12.119482496194825
Arkansas,US,2020-04-21 23:40:34,34.9697,-92.3731,1990,42,809,1948.0,5,76.86285233796326,27441,291,2.1105527638190953,84000005,USA,1059.8962467367085,14.623115577889447
California,US,2020-04-21 23:40: